In [ ]:
import sys
import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from scipy.interpolate import griddata

# Add scripts directory to path
sys.path.insert(0, str(Path.cwd().parent / 'scripts'))

from reproject_to_plane import PlaneReprojector

print("✓ Imports successful")

## 1. Configuration

In [ ]:
# Paths
config_dir = Path.cwd().parent / 'config' / 'cameras'
sirta_config = config_dir / 'sirta_camera.json'
orsay_config = config_dir / 'orsay_camera.json'

# Reprojection parameters
PLANE_ALTITUDE_KM = 10  # Contrail altitude
GRID_EXTENT_KM = 20     # ±20 km from camera
RESOLUTION_M = 50       # 50 meters per pixel

print(f"Plane altitude: {PLANE_ALTITUDE_KM} km")
print(f"Grid extent: ±{GRID_EXTENT_KM} km")
print(f"Resolution: {RESOLUTION_M} m/pixel")

## 2. Initialize Reprojectors

In [ ]:
print("Initializing SIRTA reprojector...")
sirta_reprojector = PlaneReprojector(
    camera_config_path=sirta_config,
    plane_altitude_m=PLANE_ALTITUDE_KM * 1000,
    plane_resolution_m=RESOLUTION_M
)

print("\nInitializing Orsay reprojector...")
orsay_reprojector = PlaneReprojector(
    camera_config_path=orsay_config,
    plane_altitude_m=PLANE_ALTITUDE_KM * 1000,
    plane_resolution_m=RESOLUTION_M
)

## 3. Create Projection Grids

In [ ]:
print("Creating projection grids...")
grid_east, grid_north = sirta_reprojector.create_plane_grid(extent_km=GRID_EXTENT_KM)
print(f"Grid shape: {grid_east.shape}")
print(f"Grid covers: [{grid_east.min()/1000:.1f}, {grid_east.max()/1000:.1f}] km (East)")
print(f"             [{grid_north.min()/1000:.1f}, {grid_north.max()/1000:.1f}] km (North)")

## 4. Load and Reproject SIRTA Images

In [ ]:
# Get SIRTA image directory
sirta_image_dir = Path('/data/common/STEREOSTUDYIPSL/Datasets/gQg5IUvV/PROJECTED')
sirta_images = sorted(list(sirta_image_dir.glob('*.jpg')))

print(f"Found {len(sirta_images)} SIRTA images")
print(f"First: {sirta_images[0].name}")
print(f"Last: {sirta_images[-1].name}")

In [ ]:
# Select sample images
NUM_SAMPLES = 3
sirta_samples = sirta_images[:NUM_SAMPLES]

for i, img_path in enumerate(sirta_samples, 1):
    print(f"\n{'='*70}")
    print(f"SIRTA Sample {i}: {img_path.name}")
    print('='*70)
    
    # Load and resize image
    img_loaded = Image.open(img_path).convert('RGB')
    if img_loaded.size != (sirta_reprojector.image_width, sirta_reprojector.image_height):
        img = np.array(img_loaded.resize(
            (sirta_reprojector.image_width, sirta_reprojector.image_height),
            Image.Resampling.LANCZOS
        ))
    else:
        img = np.array(img_loaded)
    
    # Reproject
    print("Reprojecting...")
    reprojected = sirta_reprojector.reproject_image(img, grid_east, grid_north)
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(18, 9))
    
    # Original
    axes[0].imshow(img)
    axes[0].set_title(f'SIRTA Original\n{img_path.name}', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Reprojected
    extent = GRID_EXTENT_KM
    axes[1].imshow(reprojected, 
                   extent=[-extent, extent, -extent, extent],
                   origin='lower')
    axes[1].set_title(f'SIRTA Reprojected at {PLANE_ALTITUDE_KM} km', 
                     fontsize=14, fontweight='bold')
    axes[1].set_xlabel('East (km)', fontsize=12)
    axes[1].set_ylabel('North (km)', fontsize=12)
    axes[1].grid(True, alpha=0.3, color='white')
    axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.7, linewidth=1.5)
    axes[1].axvline(x=0, color='red', linestyle='--', alpha=0.7, linewidth=1.5)
    
    plt.tight_layout()
    plt.show()
    
    print(f"✓ Processed")

## 5. Load and Reproject Orsay Images

In [ ]:
# Get Orsay image directory
orsay_image_dir = Path('/data/common/STEREOSTUDYIPSL/Datasets/OdnkTZQ8/PROJECTED')
orsay_images = sorted(list(orsay_image_dir.glob('*.jpg')))

print(f"Found {len(orsay_images)} Orsay images")
if orsay_images:
    print(f"First: {orsay_images[0].name}")
    print(f"Last: {orsay_images[-1].name}")

In [ ]:
# Select sample images
orsay_samples = orsay_images[:NUM_SAMPLES]

for i, img_path in enumerate(orsay_samples, 1):
    print(f"\n{'='*70}")
    print(f"Orsay Sample {i}: {img_path.name}")
    print('='*70)
    
    # Load and resize image
    img_loaded = Image.open(img_path).convert('RGB')
    if img_loaded.size != (orsay_reprojector.image_width, orsay_reprojector.image_height):
        img = np.array(img_loaded.resize(
            (orsay_reprojector.image_width, orsay_reprojector.image_height),
            Image.Resampling.LANCZOS
        ))
    else:
        img = np.array(img_loaded)
    
    # Reproject
    print("Reprojecting...")
    reprojected = orsay_reprojector.reproject_image(img, grid_east, grid_north)
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(18, 9))
    
    # Original
    axes[0].imshow(img)
    axes[0].set_title(f'Orsay Original\n{img_path.name}', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Reprojected
    extent = GRID_EXTENT_KM
    axes[1].imshow(reprojected, 
                   extent=[-extent, extent, -extent, extent],
                   origin='lower')
    axes[1].set_title(f'Orsay Reprojected at {PLANE_ALTITUDE_KM} km', 
                     fontsize=14, fontweight='bold')
    axes[1].set_xlabel('East (km)', fontsize=12)
    axes[1].set_ylabel('North (km)', fontsize=12)
    axes[1].grid(True, alpha=0.3, color='white')
    axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.7, linewidth=1.5)
    axes[1].axvline(x=0, color='red', linestyle='--', alpha=0.7, linewidth=1.5)
    
    plt.tight_layout()
    plt.show()
    
    print(f"✓ Processed")

## 6. Summary

This notebook demonstrates:
1. Loading camera calibration configurations
2. Computing azimuth/zenith to East-North projections
3. Reprojecting fisheye images to a horizontal plane at 10 km
4. Visualizing original and reprojected images

Both cameras are now projected to the same coordinate system, enabling direct comparison and stereo analysis.